# AI Video Tool on Google Colab

## トラブルシューティング
**"Bad credentials" (401 Error)** が出る場合:
あなたのGitHubトークンが無効です。
1. 新しいトークンを作成: [Tokens (classic)](https://github.com/settings/tokens/new) (Scope: `repo`)
2. Colab Secrets (`GITHUB_TOKEN`) を更新。
**重要**: Secretsを更新した後、上部メニューの「ランタイム」→「**セッションの再開**」を行わないと反映されないことがあります。

In [ ]:
# 1. Setup & Auth
import os
import shutil
import subprocess
from google.colab import userdata
from getpass import getpass

REPO_USER = "itsukikanai"
REPO_NAME = "ai-video-tool"
REPO_URL_BASE = f"github.com/{REPO_USER}/{REPO_NAME}.git"

# 1. Get Token
token = None
try:
    token = userdata.get('GITHUB_TOKEN')
except:
    pass

# 2. Clone Retries
def try_clone(url, description):
    print(f"Attempting clone ({description})...")
    if os.path.exists(REPO_NAME):
        shutil.rmtree(REPO_NAME)
    
    # Use subprocess run to capture output
    res = subprocess.run(["git", "clone", url], capture_output=True, text=True)
    if res.returncode == 0:
        print("Clone successful!")
        return True
    else:
        print(f"Failed ({description}).")
        # Print error but mask token if present
        err = res.stderr
        if token:
            err = err.replace(token, "***TOKEN***")
        print(err)
        return False

success = False

# Attempt 1: Authenticated Clone
if token:
    clone_url_auth = f"https://{REPO_USER}:{token}@{REPO_URL_BASE}"
    if try_clone(clone_url_auth, "Authenticated"):
        success = True
    else:
        print("Auth failed. Token might be invalid or expired.")

# Attempt 2: Public Clone (Fallback)
if not success:
    print("Trying public clone (in case repo is public)...")
    clone_url_public = f"https://{REPO_URL_BASE}"
    if try_clone(clone_url_public, "Public"):
        success = True

# Attempt 3: Manual Input Fallback
if not success:
    print("\nCould not clone automatically.")
    print("Repository seems Private and provided token was invalid.")
    manual_token = getpass("Please enter a valid GitHub Token (classic) manually: ")
    if manual_token:
        clone_url_manual = f"https://{REPO_USER}:{manual_token}@{REPO_URL_BASE}"
        if try_clone(clone_url_manual, "Manual Input"):
            success = True

if success:
    %cd {REPO_NAME}
    !git config user.email "colab@example.com"
    !git config user.name "Colab User"
    
    print("Installing dependencies...")
    !apt-get update && apt-get install -y ffmpeg
    !pip install -r requirements.txt
else:
    raise RuntimeError("Failed to access repository. Please check your token.")

In [ ]:
# 2. Set up Gemini API Key
import os
from google.colab import userdata

api_key = None
try:
    api_key = userdata.get('API_KEY_GEMINI')
except:
    pass

if not api_key:
    print("Please enter your Google Gemini API Key below:")
    api_key = input("API Key: ")

os.environ["API_KEY_GEMINI"] = api_key

In [ ]:
# 3. Run the Application
import sys
import os

# Ensure the current directory is in the Python path
sys.path.append(os.getcwd())

from app.ui import create_ui

print("Starting AI Video Tool... Please wait for the public URL.")

# Create the Gradio interface
try:
    demo.close()
except:
    pass
    
demo = create_ui()

# Launch with public access (share=True)
demo.launch(share=True, allowed_paths=[os.path.abspath(".")])